# Designing Twitter Search

We'll design a service that can effectively store and query user tweets.

## 1. Requirements and System Goals
- Assume Twitter has 1.5 billion total users with 800 million daily active users.
- On average Twitter gets 400 million tweets every day.
- Average size of a tweet is 300 bytes.
- Assume 500M searches a day.
- The search query will consist of multiple words combined with AND/OR.


## 2. Capacity Estimation and Constraints

Since we have 400 million new tweets each day, and each tweet on average is 300 bytes, 
the total storage we'll need is:

```
    400M * 300 =>  120GB/day
    
    Total storage per second:
        120 GB / (24 hours / 3600 seconds)  ~= 1.38MB/second
```


## 3. System APIs
We can have REST APIs expose the functionality of our service.

```python

search(
    api_dev_key: string,  # The API developer key of a registered account, this will be used for things like throttling users based on their allocated quota.
    search_terms: string,  # A string containing the search terms.
    max_results_to_return: number,  # Number of tweets to return.
    sort: number,  # optional sort mode: Last first(0 - default), Best mached (1), Most liked (2)
    page_token: string,  # This token specifies a page in the result set that should be returned.
)
```
Returns: (JSON)
```
A JSON containing info about a list of tweets matching the search query.
Each result entry can have the user ID & name, tweet text, tweet ID, creation time, number of likes, etc.
```

